In [ ]:
!pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import backtrader as bt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Define the Sentiment indicator class for Backtrader
class Sentiment(bt.Indicator):
    """Custom indicator to track sentiment scores"""
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.5,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0]
    )

    def next(self):
        self.sentiment = 0.0
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment

In [ ]:
# Define the trading strategy class
class SentimentStrategy(bt.Strategy):
    """Trading strategy based on sentiment analysis"""
    params = (
        ('period', 15),  # Moving average period
        ('printlog', True),
    )

    def __init__(self):
        # Initialize variables
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add simple moving average indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)

        # Add sentiment indicator
        self.sentiment = None
        Sentiment(self.data)
        self.plotinfo.plot = False

    def notify_order(self, order):
        """Handle order notifications"""
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        """Define trading logic"""
        date = bt.num2date(self.data.datetime[0]).date()
        prev_sentiment = self.sentiment

        if date in date_sentiment:
            self.sentiment = date_sentiment[date]

        # Check if we have an open order
        if self.order:
            return

        # Trading logic based on sentiment changes
        if not self.position and prev_sentiment:
            # Buy if price > SMA and sentiment increased significantly
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log(f'Previous Sentiment {prev_sentiment:.2f}, New Sentiment {self.sentiment:.2f} BUY CREATE, {self.dataclose[0]:.2f}')
                self.order = self.buy()

        elif prev_sentiment:
            # Sell if price < SMA and sentiment decreased significantly
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log(f'Previous Sentiment {prev_sentiment:.2f}, New Sentiment {self.sentiment:.2f} SELL CREATE, {self.dataclose[0]:.2f}')
                self.order = self.sell()

    def log(self, txt, dt=None, doprint=False):
        """Logging function"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

In [ ]:
# Function to run the trading strategy
def run_strategy(ticker, start_date, end_date, initial_cash=100000.0):
    """
    Run the trading strategy for a given ticker

    Parameters:
    ticker (str): Stock ticker symbol
    start_date (str): Start date in 'YYYY-MM-DD' format
    end_date (str): End date in 'YYYY-MM-DD' format
    initial_cash (float): Initial investment amount
    """

    # Create Backtrader engine
    cerebro = bt.Cerebro()

    # Add strategy
    cerebro.addstrategy(SentimentStrategy)

    # Get stock data
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date)

    # Add data feed
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)

    # Set initial cash
    cerebro.broker.setcash(initial_cash)

    # Set position size
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # Print initial portfolio value
    print(f'Starting Portfolio Value: ${initial_cash:.2f}')

    # Run strategy
    cerebro.run()

    # Plot results
    cerebro.plot(volume=False, style='candlestick')

    # Get final portfolio value
    final_value = cerebro.broker.getvalue()
    profit = final_value - initial_cash

    print(f'Final Portfolio Value: ${final_value:.2f}')
    print(f'Profit/Loss: ${profit:.2f}')

    return float(df['Close'][0]), profit

In [ ]:
if __name__ == "__main__":
    # Define trading parameters
    TICKER = 'AAPL'
    START_DATE = '2020-01-01'
    END_DATE = '2023-12-31'
    INITIAL_CASH = 100000.0

    dates = pd.date_range(start=START_DATE, end=END_DATE)
    date_sentiment = {date.date(): np.random.uniform(-1, 1) for date in dates}

    # Run strategy
    initial_price, profit = run_strategy(TICKER, START_DATE, END_DATE, INITIAL_CASH)

    # Print results
    print(f"\nSummary:")
    print(f"Initial {TICKER} price: ${initial_price:.2f}")
    print(f"Total profit/loss: ${profit:.2f}")
    print(f"Return on Investment: {(profit/INITIAL_CASH)*100:.2f}%")

Starting Portfolio Value: $100000.00
2020-02-05, Previous Sentiment -0.75, New Sentiment 0.36 BUY CREATE, 78.00
2020-02-06, BUY EXECUTED, Price: 78.27, Cost: 7826.75, Comm: 0.00
2020-02-21, Previous Sentiment 0.24, New Sentiment -0.86 SELL CREATE, 76.14
2020-02-24, SELL EXECUTED, Price: 72.30, Cost: 7826.75, Comm: 0.00
2020-03-31, Previous Sentiment -0.99, New Sentiment 0.37 BUY CREATE, 61.85
2020-04-01, BUY EXECUTED, Price: 59.95, Cost: 5995.20, Comm: 0.00
2020-04-02, Previous Sentiment 0.63, New Sentiment -0.70 SELL CREATE, 59.57
2020-04-03, SELL EXECUTED, Price: 59.05, Cost: 5995.20, Comm: 0.00
2020-04-08, Previous Sentiment -0.88, New Sentiment 0.16 BUY CREATE, 64.71
2020-04-09, BUY EXECUTED, Price: 65.35, Cost: 6535.14, Comm: 0.00
2020-07-29, Previous Sentiment 0.84, New Sentiment -0.57 SELL CREATE, 92.71
2020-07-30, SELL EXECUTED, Price: 91.88, Cost: 6535.14, Comm: 0.00
2020-07-31, Previous Sentiment -0.32, New Sentiment 0.36 BUY CREATE, 103.66
2020-08-03, BUY EXECUTED, Price: 10

<IPython.core.display.Javascript object>

Final Portfolio Value: $112409.50
Profit/Loss: $12409.50

Summary:
Initial AAPL price: $72.88
Total profit/loss: $12409.50
Return on Investment: 12.41%
